420-A52-SF - Algorithmes d'apprentissage supervisé - Hiver 2020 - Spécialisation technique en Intelligence Artificielle<br/>
MIT License - Copyright (c) 2020 Mikaël Swawola
<br/>
![Travaux Pratiques - Comparaison kNN et régression logistique](static/11-tp-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif la comparaison des deux types de classificateurs vus en cours jusqu'à présent, à savoir la régression logistique et la classification kNN. Leur mise en oeuvre sera réalisée à l'aide de la librairie **scikit-learn** et la notion de **compromis biais-variance** sera étudiée. Le travail sera basé sur le jeu de données **Heart** entrevu précédemment

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 0 - Chargement des bibliothèques

In [ ]:
# Manipulation de données
import numpy as np
import pandas as pd

# Visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns

# Helpers pour l'affichage
from helpers import plot_classifier

# Outils divers
from tqdm import tqdm
from collections import defaultdict

# Machine Learning
# Compléter au fur et à mesure du TP l'importation des modules scikit-learn requis

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Configuration de la visualisation
sns.set(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

## 1 - Lecture du jeu de données

<strong style="color: #4a86e8">Exercice 1-1: lire le fichier `Heart.csv`<strong/>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

cols = ['Age','Sex','ChestPain','RestBP','Chol','Fbs','RestECG','MaxHR','ExAng','Oldpeak','Slope','Ca','Thal','AHD']
HRT = pd.read_csv('../../data/Heart.csv', usecols=cols)

In [ ]:
# On supprime tout de suite les données manquantes. Ceci sera vu plus en détail plus tard dans le cours
HRT = HRT.dropna()

<strong style="color: #4a86e8">Exercice 1-2: afficher les dix premières lignes de la trame de données HRT</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

HRT.head()

## 2 - Visualisation des données

La première étape avant toute étude d'apprentissage supervisé consiste à (essayer de) visualiser les données (voir cours de *Visualisation des données*). Cette étape est faite ici pour vous.

Remarque: l'affichage ci-dessous peut prendre quelques dizaines de secondes ...

In [ ]:
g = sns.PairGrid(HRT, hue="AHD")
g = g.map(plt.scatter)

Pour la suite de l'exercice, nous allons considérer uniquement les variables explicatives `Age` et `MaxHR`, ainsi que bien évidemment la variable réponse `AHD`

<strong style="color: #4a86e8">Exercice 2-1: créer une nouvelle trame de données ne contenant que les variables `Age`, `MaxHR` et `AHD`</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

HRT2 = HRT[['Age','MaxHR','AHD']]

<strong style="color: #4a86e8">Exercice 2-2: afficher les dix premières lignes de la nouvelle trame de données `HRT2`</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

HRT2.head()

## 3 - Affichage de HRT2

In [ ]:
sns.set(style="darkgrid", rc={'figure.figsize':(12,10)})
sns.scatterplot(x='Age', y='MaxHR', hue="AHD", data=HRT2, s=70)

## 4 - Préparation des données en vue d'effectuer la validation des résultats des classificateurs

<strong style="color: #4a86e8">Exercice 4-1: extraire les variables explicatives dans un numpy array `X` et la variable réponse dans un vecteur `y`</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2 lignes

X = HRT2[['Age','MaxHR']].values
y = (HRT['AHD'] == "Yes").astype(int)

<strong style="color: #4a86e8">Exercice 4-2: à l'aide de scikit-learn, sépararer les données en jeu d'entraînement et jeu de test. La taille du jeu de test doit représenter 30% de la taille du jeu de données et l'état du générateur aléatoire sera fixé à 2020 afin de permettre la reproductibilité</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2 lignes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2020)

In [ ]:
print(f'X_train: {X_train.shape[0]} observations')
print(f'X_test: {X_test.shape[0]} observations')

## 5 - Régression logistique simple

<strong style="color: #4a86e8">Exercice 5-1: construisez la matrice des prédicteurs `X1` pour une régression logistique simple. N'oubliez pas d'inclure inclure le bias (ou intercept)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X1 = np.hstack((np.ones((X_train.shape[0],1)), X_train))

<strong style="color: #4a86e8">Exercice 5-2: à l'aide de scikit-learn, effectuez la standardisation des données</strong>

In [ ]:
# Compléter le code ci-dessous ~ 3 lignes

scaler = StandardScaler()
scaler.fit(X1)
X1_scaled = scaler.transform(X1)

<strong style="color: #4a86e8">Exercice 5-3: à l'aide de scikit-learn, entraînez un modèle de régression logistique. Puisque le biais (ou intercept) est déjà inclus dans la matrice `X1_scaled`, il ne sera pas nécéssaire de l'inclure ici. Désactivez aussi la régularisation.</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf1 = LogisticRegression(penalty="none", fit_intercept=False).fit(X1_scaled, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, clf1, scaler, order=1, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 5-4: évaluer les performances (accuracy) sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf1.score(X1_scaled, y_train)

<strong style="color: #4a86e8">Exercice 5-5: évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

X1_test = np.hstack((np.ones((X_test.shape[0],1)), X_test))
X1_test_scaled = scaler.transform(X1_test)
clf1.score(X1_test_scaled,y_test)

## 6 - Régression logistique avec variables d'interaction et variables polynomiales d'ordre 2

<strong style="color: #4a86e8">Exercice 6-1: à l'aide de scikit-learn, construisez la matrice des prédicteurs `X2` en incluant les variables d'interactions et les variables polynomiales d'ordre 2. Vous devrez également inclure le biais (ou intercept)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

poly2 = PolynomialFeatures(2)
X2 = poly2.fit_transform(X_train)

<strong style="color: #4a86e8">Exercice 6-2: à l'aide de scikit-learn, standardisez la matrice des prédicteurs</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-3 lignes

scaler = StandardScaler()
scaler.fit(X2)
X2_scaled = scaler.transform(X2)

<strong style="color: #4a86e8">Exercice 6-3:  l'aide de scikit-learn, entraînez un modèle de régression logistique. Puisque le biais (ou intercept) est déjà inclus dans la matrice `X2_scaled`, il ne sera pas nécéssaire de l'inclure ici. Comme précédemment, désactivez la régularisation.</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf2 = LogisticRegression(penalty="none", fit_intercept=False).fit(X2_scaled, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, clf2, scaler, order=2, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 6-4: évaluer les performances sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf2.score(X2_scaled, y_train)

<strong style="color: #4a86e8">Exercice 6-5: évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

X2_test = poly2.fit_transform(X_test)
X2_test_scaled = scaler.transform(X2_test)
clf2.score(X2_test_scaled,y_test)

## 7 - Régression logistique avec variables d'interaction et variables polynomiales d'ordre 3

<strong style="color: #4a86e8">Exercice 7-1: construisez la matrice des prédicteurs `X3` incluant les variables d'interactions et les variables polynomiales d'ordre 3. Vous devrez également inclure le biais (ou intercept)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

poly3 = PolynomialFeatures(3)
X3 = poly3.fit_transform(X_train)

<strong style="color: #4a86e8">Exercice 7-2: standardisez la matrice des prédicteurs</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-3 lignes

scaler = StandardScaler()
scaler.fit(X3)
X3_scaled = scaler.transform(X3)

<strong style="color: #4a86e8">Exercice 7-3: entraînez un modèle de régression logistique. Puisque le biais (ou intercept) est déjà inclus dans la matrice `X3_scaled`, il ne sera pas nécéssaire de l'inclure ici. Comme précédemment, désactivez la régularisation. Quel paramètre supplémentaire devez-vous changer ?</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf3 = LogisticRegression(penalty="none", fit_intercept=False, max_iter=1000).fit(X3_scaled, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, clf3, scaler, order=3, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 7-4: évaluer les performances sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf3.score(X3_scaled, y_train)

<strong style="color: #4a86e8">Exercice 7-5: évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

X3_test = poly3.fit_transform(X_test)
X3_test_scaled = scaler.transform(X3_test)
clf3.score(X3_test_scaled, y_test)

## 8 - Régression logistique avec variables d'interaction et variables polynomiales d'ordre 4

<strong style="color: #4a86e8">Exercice 8-1: construisez la matrice des prédicteurs `X4` incluant les variables d'interactions et les variables polynomiales d'ordre 4. Vous devrez également inclure le biais (ou intercept)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

poly4 = PolynomialFeatures(4)
X4 = poly4.fit_transform(X_train)

<strong style="color: #4a86e8">Exercice 8-2: standardisez la matrice des prédicteurs</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-3 lignes

scaler = StandardScaler()
scaler.fit(X4)
X4_scaled = scaler.transform(X4)

<strong style="color: #4a86e8">Exercice 8-3: entraînez un modèle de régression logistique. Puisque le biais (ou intercept) est déjà inclus dans la matrice `X5_scaled`, il ne sera pas nécéssaire de l'inclure ici. Comme précédemment, désactivez la régularisation.</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf4 = LogisticRegression(penalty="none", max_iter=5000).fit(X4_scaled, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, clf4, scaler, order=4, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 8-4: évaluer les performances sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf4.score(X4_scaled, y_train)

<strong style="color: #4a86e8">Exercice 8-5: évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

X4_test = poly4.fit_transform(X_test)
X4_test_scaled = scaler.transform(X4_test)
clf4.score(X4_test_scaled,y_test)

## 9 - Régression logistique avec variables d'interaction et variables polynomiales d'ordre 10

<strong style="color: #4a86e8">Exercice 9-1: construisez la matrice des prédicteurs `X10` incluant les variables d'interactions et les variables polynomiales d'ordre 10. Vous devrez également inclure le bias (ou intercept)<strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

poly10 = PolynomialFeatures(10)
X10 = poly10.fit_transform(X_train)

<strong style="color: #4a86e8">Exercice 9-2: Combien de variables la matrice X10 contient-elle ?<strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X10.shape[1]

<strong style="color: #4a86e8">Exercice 9-3: standardisez la matrice des prédicteurs</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1-3 lignes

scaler = StandardScaler()
scaler.fit(X10)
X10_scaled = scaler.transform(X10)

<strong style="color: #4a86e8">Exercice 9-4: entraînez un modèle de régression logistique. Puisque le biais (ou intercept) est déjà inclus dans la matrice `X10_scaled`, il ne sera pas nécéssaire de l'inclure ici. Comme précédemment, désactivez la régularisation.</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf10 = LogisticRegression(penalty="none", max_iter=200000).fit(X10_scaled, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, clf10, scaler, order=10, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 9-5: évaluer les performances sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

clf10.score(X10_scaled, y_train)

<strong style="color: #4a86e8">Exercice 9-6: Évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

X10_test = poly10.fit_transform(X_test)
X10_test_scaled = scaler.transform(X10_test)
clf10.score(X10_test_scaled ,y_test)

## 10 - Classification kNN

<strong style="color: #4a86e8">Exercice 10-1: à l'aide de scikit-learn, entraînez un modèle de classification KNN. Choisissez K=22</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2 lignes

neigh = KNeighborsClassifier(n_neighbors=22)
neigh.fit(X_train, y_train)

#### Affichage des résultats

In [ ]:
sns.set(rc={'figure.figsize':(16,6)})
plot_classifier(X_train, y_train, neigh, scaler=None, xlabel="Age", ylabel="maxHR")

<strong style="color: #4a86e8">Exercice 10-2: évaluer les performances sur le jeu d'entraînement</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

neigh.score(X_train, y_train)

<strong style="color: #4a86e8">Exercice 10-3: évaluer maintenant les performances sur le jeu de test</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

neigh.score(X_test,y_test)

<strong style="color: #4a86e8">Exercice 10-4: Essayer d'autres valeurs de K. Quelle est selon la valeur de K offrant un bon compromis biais-variance?</strong>

## 11 - Recherche des meilleurs hyperparamètres

### 11 - 1 Préparation des jeux de données

Nous allons maintenant considérer l'ensemble des variables explicatives du jeu de données **Heart**

<strong style="color: #4a86e8">Exercice 11-1-1: Encoder les variables explicatives catégorielles. Indice: utilisez pandas ;-)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

HRT_onehot = pd.get_dummies(HRT, columns=['ChestPain','Thal'], prefix = ['cp','thal'], drop_first=True)
X = HRT_onehot.drop(['AHD'], axis=1)
y = (HRT['AHD'] == "Yes").astype(int)

<strong style="color: #4a86e8">Exercice 11-1-2: Quel est le nombre de variables explicatives ?</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X.shape[1]

<strong style="color: #4a86e8">Exercice 11-1-3: à l'aide de scikit-learn, sépararer les données en jeu d'entraînement et jeu de test. La taille du jeu de test doit représenter 30% de la taille du jeu de données et l'état du générateur aléatoire sera fixé à 2020 afin de permettre la reproductibilité</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2020)

### 11 - 2  - Régression logistique

<strong style="color: #4a86e8">Exercice 11-2: réalisez plusieurs fois une régression logistique polynomiale (cette fois-ci SANS variables d'interaction) en faisant varier l'ordre `n` de 1 à 15. Vous pouvez utiliser la fonction `polynomial` ci-dessous. Enregistrez les scores sur les jeux d'entraînement et de test pour chaque valeur de `n` dans le dictionnaire `history`</strong>

In [ ]:
def polynomial(X, degree=2):
    vec = [np.ones((X.shape[0],1))]
    for d in range(1,degree+1):
        temp = np.array((X**d))
        vec.append(temp)
    return np.hstack(vec)

In [ ]:
history = defaultdict(list)
for n in tqdm(range(1,15)):
    # Compléter le code ci-dessous ~ quelques lignes ...
    
    # variables polynomiales
    X_temp = polynomial(X_train, degree=n)
    
    # Standardisation
    scaler = StandardScaler()
    scaler.fit(X_temp)
    X_scaled = scaler.transform(X_temp)
    
    # Régression logistique
    clf = LogisticRegression(penalty="none", solver="sag", max_iter=200000).fit(X_scaled, y_train)
    
    # Performances
    Xn_test =  polynomial(X_test, degree=n)
    X_test_scaled = scaler.transform(Xn_test)
    history['train'].append(clf.score(X_scaled, y_train))
    history['test'].append(clf.score(X_test_scaled, y_test))

#### Affichage de la courbe de validation

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1,15), history['train'], label="train")
ax.plot(range(1,15), history['test'], label="test")
ax.set_xlabel('n', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 11 - 3  - Classification kNN

<strong style="color: #4a86e8">Exercice 11-3: réalisez plusieurs fois une classification kNN en faisant varier le nombre de voisins `k` de 1 à 200. Enregistrez les scores sur les jeux d'entraînement et de test pour chaque valeur de `n` dans le dictionnaire `history`</strong>

In [ ]:
history = defaultdict(list)
for k in tqdm(range(1,200)):
    # Compléter le code ci-dessous ~ quelques lignes ...
    
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)
    history['train'].append(neigh.score(X_train, y_train))
    history['test'].append(neigh.score(X_test, y_test))

#### Affichage de la courbe de validation

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1,200),history['train'], label="train")
ax.plot(range(1,200), history['test'], label="test")
ax.set_xlabel('k', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 11 - 4 - Choix du modèle

<strong style="color: #4a86e8">Exercice 11-4: Quel modèle choisissez-vous ? Quels sont les valeurs des hyperparamètres ?</strong>

### Fin du TP